In [1]:
import os

In [2]:
%pwd

'd:\\Version Control\\AI-Text-Summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Version Control\\AI-Text-Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [13]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])
        data_transformation_config = DataTransformationConfig(
            root_dir=Path(config.root_dir),
            data_path=Path(config.data_path),
            tokenizer_name=config.tokenizer_name
        )
        return data_transformation_config

In [14]:
from transformers import AutoTokenizer
print("Transformer is good to go!")

Transformer is good to go!


In [15]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [16]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_name)

    def convert_examples_to_features(self, examples_batch):
        input_encodings = self.tokenizer(examples_batch['dialogue'], max_length=1024, truncation=True)
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(examples_batch['summary'], max_length=128, truncation=True)

        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))


In [19]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
    logger.info("Data transformation is completed!")
except Exception as e:
    logger.exception(e)
    raise e

[2025-10-22 20:17:11,442: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-22 20:17:11,447: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-22 20:17:11,449: INFO: common: created directory at: artifacts]
[2025-10-22 20:17:11,452: INFO: common: created directory at: artifacts/data_transformation]
[2025-10-22 20:17:13,877: ERROR: 4090147737: Directory artifacts\data_ingestion\samsum_dataset is neither a `Dataset` directory nor a `DatasetDict` directory.]
Traceback (most recent call last):
  File "C:\Users\aminu\AppData\Local\Temp\ipykernel_21248\4090147737.py", line 5, in <module>
    data_transformation.convert()
  File "C:\Users\aminu\AppData\Local\Temp\ipykernel_21248\1537832415.py", line 18, in convert
    dataset_samsum = load_from_disk(self.config.data_path)
  File "c:\Users\aminu\miniconda3\envs\textS\lib\site-packages\datasets\load.py", line 1479, in load_from_disk
    raise FileNotFoundError(
FileNotFoundError: Directory artifacts\data

FileNotFoundError: Directory artifacts\data_ingestion\samsum_dataset is neither a `Dataset` directory nor a `DatasetDict` directory.